In [1]:
#libraries

library(RMySQL)
library(dplyr)

Loading required package: DBI


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
#connect

con = dbConnect(MySQL(), dbname = "BnL",
                host = "mysql-1.cda.hhs.se", port = 3306,
                user = "bnl", password = "bnl@sse")

In [3]:
#Modify this to import your dataset

df = dbGetQuery(con, 
                "SELECT 
                  item, 
                  SUM(returned)/COUNT(returned) AS prop_returned, 
                  MIN(class_desc) AS class_desc,
                  MIN(group_desc) AS group_desc,
                  MIN(division_desc) AS division_desc, 
                  MIN(sustainability_id_desc) AS sustainability_id_desc,
                  purchase_date
                FROM Products 
                INNER JOIN Transactions 
                  USING (item) 
                GROUP BY item
                order by 7 asc")



Warning message in .local(conn, statement, ...):
“Decimal MySQL column 1 imported as numeric”
Warning message in .local(conn, statement, ...):
“unrecognized MySQL field type 7 in column 6 imported as character”


In [4]:
# dataset

head(df)

item,prop_returned,class_desc,group_desc,division_desc,sustainability_id_desc,purchase_date
16976458,0.0374,Eye Colour B,Makeup,Beauty,NA,2017-03-18 07:49:00
43708023,0.0000,Receipt texts,Services,Service,NA,2017-03-18 07:49:00
42641555,0.0216,Face Makeup B,Makeup,Beauty,NA,2017-03-18 07:49:00
43573146,0.0000,Hair Care B,Hair,Beauty,Non sustainable,2017-03-18 07:49:00
40832065,0.0000,Receipt texts,Services,Service,NA,2017-03-18 09:34:00
44338259,0.0000,GWP,Product related Non Merchandise,Beauty,NA,2017-03-18 09:34:00


In [5]:
summary(df)

      item          prop_returned      class_desc         group_desc       
 Min.   :  275040   Min.   :0.00000   Length:28034       Length:28034      
 1st Qu.:42955231   1st Qu.:0.00000   Class :character   Class :character  
 Median :44698359   Median :0.00000   Mode  :character   Mode  :character  
 Mean   :59374846   Mean   :0.01265                                        
 3rd Qu.:88128158   3rd Qu.:0.00000                                        
 Max.   :89378150   Max.   :1.00000                                        
 division_desc      sustainability_id_desc purchase_date     
 Length:28034       Length:28034           Length:28034      
 Class :character   Class :character       Class :character  
 Mode  :character   Mode  :character       Mode  :character  
                                                             
                                                             
                                                             

In [6]:
df = as.data.frame(unclass(df),                     # Convert all character columns to factor
                       stringsAsFactors = TRUE)
str(df)

'data.frame':	28034 obs. of  7 variables:
 $ item                  : int  16976458 43708023 42641555 43573146 40832065 44338259 44240984 43272798 21601695 43650480 ...
 $ prop_returned         : num  0.0374 0 0.0216 0 0 0 0.0154 0 0.0083 0 ...
 $ class_desc            : Factor w/ 67 levels "Accessories C",..: 9 45 15 24 45 20 15 45 13 9 ...
 $ group_desc            : Factor w/ 16 levels "Body","Childrens Acc.",..: 9 11 9 8 11 10 9 11 5 9 ...
 $ division_desc         : Factor w/ 5 levels "Beauty","Children",..: 1 4 1 1 4 1 1 4 1 1 ...
 $ sustainability_id_desc: Factor w/ 6 levels "Allergy and health",..: NA NA NA 3 NA NA NA NA NA NA ...
 $ purchase_date         : Factor w/ 21527 levels "2017-03-18 07:49:00",..: 1 1 1 1 2 2 2 2 2 2 ...


In [7]:
# imputation for missing values

sum(is.na(df$returned))
sum(is.na(df$item))
sum(is.na(df$class_desc))
sum(is.na(df$group_desc))
sum(is.na(df$division_desc))
sum(is.na(df$sustainability_id_desc))
sum(is.na(df$purchase_date))

df <- df %>% mutate(sustainability_id_desc	 = ifelse(is.na(sustainability_id_desc	), "Sustainable", sustainability_id_desc	))

df = na.omit(df)

[1] 0

[1] 0

[1] 102

[1] 102

[1] 102

[1] 22304

[1] 0

In [8]:
head(df,10)

item,prop_returned,class_desc,group_desc,division_desc,sustainability_id_desc,purchase_date
16976458,0.0374,Eye Colour B,Makeup,Beauty,Sustainable,2017-03-18 07:49:00
43708023,0.0000,Receipt texts,Services,Service,Sustainable,2017-03-18 07:49:00
42641555,0.0216,Face Makeup B,Makeup,Beauty,Sustainable,2017-03-18 07:49:00
43573146,0.0000,Hair Care B,Hair,Beauty,3,2017-03-18 07:49:00
40832065,0.0000,Receipt texts,Services,Service,Sustainable,2017-03-18 09:34:00
44338259,0.0000,GWP,Product related Non Merchandise,Beauty,Sustainable,2017-03-18 09:34:00
44240984,0.0154,Face Makeup B,Makeup,Beauty,Sustainable,2017-03-18 09:34:00
43272798,0.0000,Receipt texts,Services,Service,Sustainable,2017-03-18 09:34:00
21601695,0.0083,Face Cleansing B,Face,Beauty,Sustainable,2017-03-18 09:34:00
43650480,0.0000,Eye Colour B,Makeup,Beauty,Sustainable,2017-03-18 09:34:00


In [9]:
summary(df)

      item          prop_returned                  class_desc  
 Min.   :  275040   Min.   :0.00000   Lips B            :4172  
 1st Qu.:42955134   1st Qu.:0.00000   Face Makeup B     :3927  
 Median :44697898   Median :0.00000   Eye Colour B      :3369  
 Mean   :59348261   Mean   :0.01266   Nails B           :3071  
 3rd Qu.:88134193   3rd Qu.:0.00000   Face Care B       :2405  
 Max.   :89378150   Max.   :1.00000   Fragrance Female B:1864  
                                      (Other)           :9124  
              group_desc           division_desc   sustainability_id_desc
 Makeup            :16663   Beauty        :26603   Length:27932          
 Face              : 3072   Children      :   12   Class :character      
 Fragrance         : 2785   Home          :   14   Mode  :character      
 Body              : 2195   Service       :  120                         
 Hair              : 1244   Womens Fashion: 1183                         
 Womens Accessories: 1141                   

In [10]:
#First, we split the dataset into a training set and a test set.

set.seed(7313)   # set seed to ensure you always have same random numbers generated
n = nrow(df)
p = 0.7
train = sample(n, p*n)

In [11]:
#Splitting by time

nrow(df)
27932 * .7

train_time  <- df[1:19552, ]
test_time   <- df[19553:28034, ]

[1] 27932

[1] 19552.4

In [12]:
#Preview of train set split by time

head(train_time,10)

item,prop_returned,class_desc,group_desc,division_desc,sustainability_id_desc,purchase_date
16976458,0.0374,Eye Colour B,Makeup,Beauty,Sustainable,2017-03-18 07:49:00
43708023,0.0000,Receipt texts,Services,Service,Sustainable,2017-03-18 07:49:00
42641555,0.0216,Face Makeup B,Makeup,Beauty,Sustainable,2017-03-18 07:49:00
43573146,0.0000,Hair Care B,Hair,Beauty,3,2017-03-18 07:49:00
40832065,0.0000,Receipt texts,Services,Service,Sustainable,2017-03-18 09:34:00
44338259,0.0000,GWP,Product related Non Merchandise,Beauty,Sustainable,2017-03-18 09:34:00
44240984,0.0154,Face Makeup B,Makeup,Beauty,Sustainable,2017-03-18 09:34:00
43272798,0.0000,Receipt texts,Services,Service,Sustainable,2017-03-18 09:34:00
21601695,0.0083,Face Cleansing B,Face,Beauty,Sustainable,2017-03-18 09:34:00
43650480,0.0000,Eye Colour B,Makeup,Beauty,Sustainable,2017-03-18 09:34:00


In [13]:
#After splitting data, we provide a glimpse of the training set

summary(train)
summary(train_time)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      1    6994   13930   13944   20930   27931 

      item          prop_returned                  class_desc  
 Min.   :  275040   Min.   :0.00000   Lips B            :2768  
 1st Qu.:40894226   1st Qu.:0.00000   Face Makeup B     :2677  
 Median :44042856   Median :0.00000   Eye Colour B      :2286  
 Mean   :50416012   Mean   :0.01272   Nails B           :2246  
 3rd Qu.:75253584   3rd Qu.:0.00000   Face Care B       :1762  
 Max.   :88343530   Max.   :1.00000   Fragrance Female B:1323  
                                      (Other)           :6490  
              group_desc           division_desc   sustainability_id_desc
 Makeup            :11570   Beauty        :18623   Length:19552          
 Face              : 2270   Children      :    8   Class :character      
 Fragrance         : 1986   Home          :    9   Mode  :character      
 Body              : 1654   Service       :  112                         
 Hair              :  992   Womens Fashion:  800                         
 Womens Accessories:  772                   